In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

In [14]:
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3_align = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)
img_3_align.fill(255)
cv2.namedWindow("ws6")

In [15]:
def stitch_image(images = []):
    if len(images) == 1:
        print(type(images[0]))
        return images[0]
    MIN_MATCH_COUNT = 4
    print(f"Number of image: {len(images)}")
    images_gray = []
    images_keypoints = []
    images_descriptions = []
    bf = cv2.BFMatcher()
    sift = cv2.SIFT_create()
    for i in range(len(images)):
        images_gray.append(cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY))
        kp_temp, des_temp = sift.detectAndCompute(images[i],None)
        images_keypoints.append(kp_temp)
        images_descriptions.append(des_temp)
    matchesES =  [0]
    len_of_good_for_each_image = [0]
    lst_of_good_kp_of_each_matches = [0]
    for i in range(1, len(images)):
        matchesES.append(bf.knnMatch(images_descriptions[0],images_descriptions[i], k = 2))
        good_for_each_matches = []
        for m,n in matchesES[i]:
            if m.distance < .8*n.distance:
                good_for_each_matches.append(m)
        len_of_good_for_each_image.append(len(good_for_each_matches))
        lst_of_good_kp_of_each_matches.append(good_for_each_matches)
    index_of_most_match_image_with_image_0 = len_of_good_for_each_image.index(max(len_of_good_for_each_image)) 
    good = lst_of_good_kp_of_each_matches[index_of_most_match_image_with_image_0]
    kp_ref = images_keypoints[0]
    kp_align = images_keypoints[index_of_most_match_image_with_image_0]
    align_image = images[index_of_most_match_image_with_image_0]
    reference_image = images[0]
    reference_image_grey = images_gray[index_of_most_match_image_with_image_0]
    # check left right
    reference_image_grey_left = reference_image_grey[:, :reference_image_grey.shape[1]//2]
    reference_image_grey_right = reference_image_grey[:, reference_image_grey.shape[1]//2:]
    kp_left, des_left = sift.detectAndCompute(reference_image_grey_left,None)
    kp_right, des_right = sift.detectAndCompute(reference_image_grey_right,None)
    kp_align, des_align = sift.detectAndCompute(align_image,None)

    matches_left = bf.knnMatch(des_left,des_align, k = 2)
    matches_right = bf.knnMatch(des_right,des_align, k = 2)
    left = []
    for m,n in matches_left:
        if m.distance < .75*n.distance:
            left.append(m)
    right = []
    for m,n in matches_right:
        if m.distance < .75*n.distance:
            right.append(m)
    if len(left) < len(right):
        reference_image, align_image = align_image, reference_image


    # find the two image that have the most common keypoint
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp_ref[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp_align[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(dst_pts,src_pts, cv2.RANSAC)
        matchesMask = mask.ravel().tolist()
        h,w = reference_image_grey.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        #dst = cv2.perspectiveTransform(pts,M)
        aligned_image = cv2.warpPerspective(align_image, M, (reference_image.shape[1]+ align_image.shape[1], align_image.shape[0]))
        cv2.imshow(f'aligned_image', aligned_image)


        mask = 255 * np.ones(reference_image.shape, reference_image.dtype)
        center = (reference_image.shape[1]//2, reference_image.shape[0]//2)
        result = cv2.seamlessClone(reference_image, aligned_image, mask, center, cv2.NORMAL_CLONE)
        cv2.imshow('result', aligned_image)




        # aligned_image[0:align_image.shape[0], 0:align_image.shape[1]] = align_image
        # aligned_image[0:reference_image.shape[0], 0:reference_image.shape[1]] = reference_image
        print(aligned_image.shape)
    else:
        print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    img_match = cv2.drawMatches(reference_image,kp_ref,align_image,kp_align,good,None,**draw_params)
    cv2.imshow('img_match' + str(random.randint(1,120)), img_match)
    images.pop(index_of_most_match_image_with_image_0)
    images.pop(0)
    images.insert(0, result)
    return stitch_image(images)

In [16]:
images = []
while(True):
    cv2.imshow('ws6', img_3)
    k = cv2.waitKey(0)
    if k == ord('f'):
        for i in range(2):
            image = cv2.imread(str(i+1) + '.jpg')
            images.append(image)
        img_match_all= stitch_image(images)
        print(type(img_match_all))
        cv2.imshow('img_match_all', img_match_all)
    if k == ord('l'): # load
        Num_of_image = int(input('How many image do you want to stitch?'))
        for i in range(Num_of_image):
            image_path = input(f'image {i} path: ')
            image = cv2.imread(image_path)
            cv2.imshow(f'image {i}', image)
            images.append(image)
    if k == 13: # enter key
        img_match_all= stitch_image(images)
        cv2.imshow('img_match_all', img_match_all)
    if (k == ord('q')): # quit
        cv2.destroyAllWindows()
        break

Number of image: 2
(640, 960, 3)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
